In this notebook we will load and use a question answering language model that can retrieve the answer to a question from a given text. Learn more about the Question Answering task <a href="https://huggingface.co/tasks/question-answering" target="_blank" rel="noopener">here</a>.

We will be running SQL queries using <a href="https://jupysql.ploomber.io/en/latest/quick-start.html" target="_blank" rel="noopener"> JupySQL</a> SQL Magic.

Prior to using this notebook one needs to complete the follow steps:
1. [Create the database schema](../setup_db.ipynb).
2. [Initialize the Transformer Extension](te_init.ipynb).

In [5]:
# TODO: Move this to a separate configuration notebook. Here we just need to load this configuration from a store.
from dataclasses import dataclass

@dataclass
class SandboxConfig:
    EXTERNAL_HOST_NAME = "192.168.124.93"
    HOST_PORT = "8888"

    @property
    def EXTERNAL_HOST(self):
        return f"""{self.EXTERNAL_HOST_NAME}:{self.HOST_PORT}"""

    USER = "sys"
    PASSWORD = "exasol"
    BUCKETFS_PORT = "6666"
    BUCKETFS_USER = "w"
    BUCKETFS_PASSWORD = "write"
    BUCKETFS_USE_HTTPS = False
    BUCKETFS_SERVICE = "bfsdefault"
    BUCKETFS_BUCKET = "default"

    @property
    def EXTERNAL_BUCKETFS_HOST(self):
        return f"""{self.EXTERNAL_HOST_NAME}:{self.BUCKETFS_PORT}"""

    @property
    def BUCKETFS_URL_PREFIX(self):
        return "https://" if self.BUCKETFS_USE_HTTPS else "http://"

    @property
    def BUCKETFS_PATH(self):
        # Filesystem-Path to the read-only mounted BucketFS inside the running UDF Container
        return f"/buckets/{self.BUCKETFS_SERVICE}/{self.BUCKETFS_BUCKET}"

    SCRIPT_LANGUAGE_NAME = "PYTHON3_60"
    UDF_FLAVOR = "python3-ds-EXASOL-6.0.0"
    UDF_RELEASE= "20190116"
    UDF_CLIENT = "exaudfclient" # or for newer versions of the flavor exaudfclient_py3
    SCHEMA = "IDA"

    @property
    def SCRIPT_LANGUAGES(self):
        return f"""{self.SCRIPT_LANGUAGE_NAME}=localzmq+protobuf:///{self.BUCKETFS_SERVICE}/
            {self.BUCKETFS_BUCKET}/{self.UDF_FLAVOR}?lang=python#buckets/{self.BUCKETFS_SERVICE}/
            {self.BUCKETFS_BUCKET}/{self.UDF_FLAVOR}/exaudf/{self.UDF_CLIENT}""";

    @property
    def connection_params(self):
        return {"dns": self.EXTERNAL_HOST, "user": self.USER, "password": self.PASSWORD, "compression": True}

    @property
    def params(self):
        return {
            "script_languages": self.SCRIPT_LANGUAGES,
            "script_language_name": self.SCRIPT_LANGUAGE_NAME,
            "schema": self.SCHEMA,
            "BUCKETFS_PORT": self.BUCKETFS_PORT,
            "BUCKETFS_USER": self.BUCKETFS_USER,
            "BUCKETFS_PASSWORD": self.BUCKETFS_PASSWORD,
            "BUCKETFS_USE_HTTPS": self.BUCKETFS_USE_HTTPS,
            "BUCKETFS_BUCKET": self.BUCKETFS_BUCKET,
            "BUCKETFS_PATH": self.BUCKETFS_PATH
        }

    # Name of the BucketFS connection
    BFS_CONN = 'MyBFSConn'

    # Name of a sub-directory of the bucket root
    BFS_DIR = 'my_storage'

    # We will store all models in this sub-directory at BucketFS
    TE_MODELS_DIR = 'models'
    
    # We will save cached model in this sub-directory relative to the current directory on the local machine.
    TE_MODELS_CACHE_DIR = 'models_cache'

    @property
    def WEBSOCKET_URL(self):
        return f"exa+websocket://{self.USER}:{self.PASSWORD}@{self.EXTERNAL_HOST}/{self.SCHEMA}?SSLCertificate=SSL_VERIFY_NONE"

conf = SandboxConfig()

First let's bring up the JupySQL and connect to the database via the SQLAlchemy. Please refer to the documentation in the <a href="https://github.com/exasol/sqlalchemy-exasol" target="_blank" rel="noopener">sqlalchemy-exasol</a> for details on how to connect to the database using Exasol SQLAlchemy driver.

In [ ]:
from sqlalchemy import create_engine

engine = create_engine(conf.WEBSOCKET_URL)

%load_ext sql
%sql engine

Now we will download a model from the Huggingface Hub and put into the BucketFS.

There are two ways of doing this.
1. Using the `TE_MODEL_DOWNLOADER_UDF` UDF.
2. Downloading a model to a local drive and subsequently uploading in into the BucketFS using a CLI.

In this notebook we will use the second method.

To demonstrate the question answering task we will use a [roberta model](https://huggingface.co/deepset/roberta-base-squad2).
This is a public model, therefore the last parameter - the name of the Huggingface token connection - can be an empty string.

In [8]:
# This is the name of the model at the Huggingface Hub
MODEL_NAME = 'deepset/roberta-base-squad2'

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, cache_dir=conf.TE_MODELS_CACHE_DIR)
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME, cache_dir=conf.TE_MODELS_CACHE_DIR)

Now we can upload the model into the BucketFS using a command line. Unfortunately we cannot tell exactly when this process has finished. Notebook's hourglass may not be a reliable indicator. BucketFS will still be doing some work when the call issued by the notebook returns. Please wait for few moments after that, before querying the model.

In [9]:
upload_command = f"""python -m exasol_transformers_extension.upload_model \
    --bucketfs-name {conf.BUCKETFS_SERVICE} \
    --bucketfs-host {conf.EXTERNAL_HOST_NAME} \
    --bucketfs-port {conf.BUCKETFS_PORT} \
    --bucketfs-user {conf.BUCKETFS_USER} \
    --bucketfs-password {conf.BUCKETFS_PASSWORD} \
    --bucket {conf.BUCKETFS_BUCKET} \
    --path-in-bucket {conf.BFS_DIR} \
    --model-name {MODEL_NAME}  \
    --sub-dir {conf.TE_MODELS_DIR} \
    --local-model-path {conf.TE_MODELS_CACHE_DIR}
    """
!{upload_command}

In [10]:
# This will be our question
TEST_QUESTION = 'What bitumen is used for?'

# Let's first try it with the following context
TEST_CONTEXT1 = """
Apart from stylish design features of new flat roofs, the other thing that’s moved on considerably is the technology
used to keep them weather-proof. Once flat roofs were notoriously prone to leaking and the problem could only be solved
with a boiling cauldron of tar. These days there are patch repair kits, liquid rubber membranes and even quick,
efficient waterproofing paint that last for ages – and can even be applied in damp weather.
"""

# Make sure our texts can be used in an SQL statement.
TEST_QUESTION = TEST_QUESTION.replace("'", "''")
TEST_CONTEXT1 = TEST_CONTEXT1.replace("'", "''")

# We will collect 5 best answers.

In [ ]:
%%sql
WITH MODEL_OUTPUT AS
(
    SELECT TE_QUESTION_ANSWERING_UDF(
        NULL,
        '{{conf.BFS_CONN}}',
        NULL,
        '{{conf.TE_MODELS_DIR}}',
        '{{MODEL_NAME}}',
        '{{TEST_QUESTION}}',
        '{{TEST_CONTEXT1}}',
        5
    )
)
SELECT answer, score, error_message FROM MODEL_OUTPUT ORDER BY SCORE DESC

Let's change the context and see a different set of answers.

In [13]:
# New context
TEST_CONTEXT2 = """
You can make a wooden planter in a day, using treated timber. Simply work out how big an area you need,
cut the wood to size and follow our steps to putting the planter together. Make sure your wooden planter
has drainage holes, so plants don’t become waterlogged.
"""

TEST_CONTEXT2 = TEST_CONTEXT2.replace("'", "''")

In [ ]:
%%sql
WITH MODEL_OUTPUT AS
(
    SELECT TE_QUESTION_ANSWERING_UDF(
        NULL,
        '{{conf.BFS_CONN}}',
        NULL,
        '{{conf.TE_MODELS_DIR}}',
        '{{MODEL_NAME}}',
        '{{TEST_QUESTION}}',
        '{{TEST_CONTEXT2}}',
        5
    )
)
SELECT answer, score, error_message FROM MODEL_OUTPUT ORDER BY SCORE DESC